In [ ]:
!gdown --id 1-cs3am9gDETODriP5-Ka_RxaXBpA7SLF -O disease.zip
import zipfile
import os

zip_file = "disease.zip"  # Name of the downloaded file

# Extract the ZIP file
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall("/content/extracted_file")  # Change path if needed

print("Extraction completed!")


/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1-cs3am9gDETODriP5-Ka_RxaXBpA7SLF
From (redirected): https://drive.google.com/uc?id=1-cs3am9gDETODriP5-Ka_RxaXBpA7SLF&confirm=t&uuid=12c02203-d32d-4a6e-851b-6e7736dfa5a6
To: /content/disease.zip
100% 2.49G/2.49G [00:32<00:00, 76.3MB/s]
Extraction completed!


In [ ]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import  ModelCheckpoint

# Label map (update if needed)
label_map = {
    '1.Dry AMD': 0,
    '2.Wet AMD': 1,
    '3.Mild DR': 2,
    '4.Moderate DR': 3,
    '5.Severe DR': 4,
    '6.Proliferate DR': 5,
    '7.Cataract': 6,
    '8.Hypertensive Retinopathy': 7,
    '9.Pathological Myopia': 8,
    '10.Glaucoma': 9,
    '11.Normal Fundus': 10,
}

# ----------------------------
# Custom Data Generator
# ----------------------------
class ImageDataGenerator(Sequence):
    def __init__(self, image_paths, labels, batch_size=32, shuffle=True, target_size=(224, 224)):
        self.image_paths = image_paths
        self.labels = labels
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.target_size = target_size
        self.indices = np.arange(len(self.image_paths))
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch_images, batch_labels = [], []

        for i in batch_indices:
            img = cv2.imread(self.image_paths[i])
            img = cv2.resize(img, self.target_size)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.equalizeHist(img)
            img = img.astype('float32') / 255.0
            img = np.expand_dims(img, axis=-1)
            batch_images.append(img)
            batch_labels.append(self.labels[i])

        return np.array(batch_images), np.array(batch_labels)

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

# ----------------------------
# Function to collect image paths and labels
# ----------------------------
def get_image_paths_and_labels(base_folder):
    image_paths = []
    labels = []
    for label_name, label_value in label_map.items():
        folder_path = os.path.join(base_folder, label_name)
        if not os.path.exists(folder_path):
            print(f"Warning: Folder {folder_path} not found. Skipping.")
            continue
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            if os.path.isfile(img_path):
                image_paths.append(img_path)
                labels.append(label_value)
    return image_paths, labels

# ----------------------------
# CNN Model Definition
# ----------------------------
def build_ImprovedCNN(input_shape=(224, 224, 1)):
    model = Sequential([
        Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Flatten(),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dense(64, activation='relu'),
        Dense(11, activation='softmax')  # 11 classes
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# ----------------------------
# Load data and create generators
# ----------------------------
base_folder = "/content/extracted_file1/Retinal Fundus Images/train"
image_paths, labels = get_image_paths_and_labels(base_folder)

# Train-test split
train_paths, test_paths, train_labels, test_labels = train_test_split(
    image_paths, labels, test_size=0.1, random_state=42, stratify=labels)

# Generators
train_generator = ImageDataGenerator(train_paths, train_labels, batch_size=32)
test_generator = ImageDataGenerator(test_paths, test_labels, batch_size=32, shuffle=False)


callbacks = [
    ModelCheckpoint("best_model_all.h5", save_best_only=True),
]
# ----------------------------
# Train and Evaluate
# ----------------------------
DenseCNN_model = build_ImprovedCNN()
DenseCNN_model.fit(train_generator, epochs=40, validation_data=test_generator,callbacks=callbacks)

# Evaluate model
test_loss, test_accuracy = DenseCNN_model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/40
565/565 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step - accuracy: 0.5620 - loss: 1.1350

565/565 ━━━━━━━━━━━━━━━━━━━━ 128s 202ms/step - accuracy: 0.5621 - loss: 1.1346 - val_accuracy: 0.3586 - val_loss: 2.0289
Epoch 2/40
565/565 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step - accuracy: 0.7165 - loss: 0.6596

565/565 ━━━━━━━━━━━━━━━━━━━━ 107s 189ms/step - accuracy: 0.7165 - loss: 0.6596 - val_accuracy: 0.5438 - val_loss: 1.1154
Epoch 3/40
565/565 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - accuracy: 0.7855 - loss: 0.4970

565/565 ━━━━━━━━━━━━━━━━━━━━ 102s 181ms/step - accuracy: 0.7855 - loss: 0.4970 - val_accuracy: 0.6524 - val_loss: 0.8203
Epoch 4/40
565/565 ━━━━━━━━━━━━━━━━━━━━ 101s 179ms/step - accuracy: 0.8466 - loss: 0.3706 - val_accuracy: 0.6474 - val_loss: 0.9039
Epoch 5/40
565/565 ━━━━━━━━━━━━━━━━━━━━ 98s 173ms/step - accuracy: 0.9357 - loss: 0.1734 - val_accuracy: 0.6823 - val_loss: 1.1943
Epoch 6/40
565/565 ━━━━━━━━━━━━━━━━━━━━ 100s 176ms/step - accuracy: 0.9728 - loss: 0.0881 - val_accuracy: 0.6330 - val_loss: 1.3641
Epoch 7/40
565/565 ━━━━━━━━━━━━━━━━━━━━ 98s 173ms/step - accuracy: 0.9844 - loss: 0.0523 - val_accuracy: 0.6823 - val_loss: 1.4405
Epoch 8/40
565/565 ━━━━━━━━━━━━━━━━━━━━ 99s 174ms/step - accuracy: 0.9328 - loss: 0.1953 - val_accuracy: 0.7176 - val_loss: 1.1226
Epoch 9/40
565/565 ━━━━━━━━━━━━━━━━━━━━ 97s 172ms/step - accuracy: 0.9905 - loss: 0.0330 - val_accuracy: 0.6917 - val_loss: 1.1659
Epoch 10/40
565/565 ━━━━━━━━━━━━━━━━━━━━ 99s 175ms/step - accuracy: 0.9957 - loss: 0.0184 -

KeyboardInterrupt: 